<a href="https://colab.research.google.com/github/elangbijak4/blockchain-codes-use-generative-AI/blob/main/Blockchain_DPoS-PoS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import hashlib
import time
import random

class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

    def __repr__(self):
        return f"Block(Index: {self.index}, Hash: {self.hash})"

class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.stakeholders = {}  # To keep track of stakeholder's stakes
        self.delegates = []  # List of selected delegates for DPoS

    def create_genesis_block(self):
        return Block(0, "0", time.time(), "Genesis Block", self.calculate_hash(0, "0", time.time(), "Genesis Block"))

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.hash = self.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)

    def calculate_hash(self, index, previous_hash, timestamp, data):
        value = str(index) + previous_hash + str(timestamp) + data
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    def add_stakeholder(self, address, stake):
        if address in self.stakeholders:
            self.stakeholders[address] += stake
        else:
            self.stakeholders[address] = stake

    def select_validator(self):
        total_stake = sum(self.stakeholders.values())
        pick = random.uniform(0, total_stake)
        current = 0
        for address, stake in self.stakeholders.items():
            current += stake
            if current > pick:
                return address

    def select_delegates(self, num_delegates):
        sorted_stakeholders = sorted(self.stakeholders.items(), key=lambda item: item[1], reverse=True)
        self.delegates = [address for address, _ in sorted_stakeholders[:num_delegates]]
        print(f"Selected Delegates: {self.delegates}")

    def create_block_pos(self, data):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = time.time()
        previous_hash = latest_block.hash
        new_block = Block(index, previous_hash, timestamp, data, "")
        validator = self.select_validator()
        print(f"Block #{index} created by {validator} using PoS")
        self.add_block(new_block)

    def create_block_dpos(self, data):
        if not self.delegates:
            raise Exception("No delegates selected for DPoS.")
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = time.time()
        previous_hash = latest_block.hash
        new_block = Block(index, previous_hash, timestamp, data, "")
        delegate = random.choice(self.delegates)
        print(f"Block #{index} created by {delegate} using DPoS")
        self.add_block(new_block)




In [2]:
# Example Usage
blockchain = Blockchain()
blockchain.add_stakeholder('Alice', 10)
blockchain.add_stakeholder('Bob', 20)
blockchain.add_stakeholder('Charlie', 30)

In [3]:
# PoS Block Creation
blockchain.create_block_pos('Block 1 Data')

Block #1 created by Charlie using PoS


In [4]:
# Selecting delegates for DPoS
blockchain.select_delegates(2)  # Select top 2 stakeholders as delegates

Selected Delegates: ['Charlie', 'Bob']


In [5]:
# DPoS Block Creation
blockchain.create_block_dpos('Block 2 Data')
blockchain.create_block_dpos('Block 3 Data')

Block #2 created by Charlie using DPoS
Block #3 created by Bob using DPoS


In [6]:
for block in blockchain.chain:
    print(block)

Block(Index: 0, Hash: 5403b8d2bff8033b9d10c5db4a4a5defd3c5262cf95bb8fc7639b88aa184263a)
Block(Index: 1, Hash: 1c6de70b7e4478c6f78e60970069dcac96f88f5defa550902802240c059c3182)
Block(Index: 2, Hash: e85e203077fafb9053bc1dbcb10b998eb1c4c8b51698fa888983e8a02631d06d)
Block(Index: 3, Hash: dd6fd43ce17a33c7df3bc62fb877c29ecab93377bfe71af23101f450e6bc74ea)
